In [1]:
#coding:utf-8
#稳定函数模块
from tqdm import tqdm
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import time
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.preprocessing import Binarizer
from sklearn.metrics import roc_auc_score,classification_report,f1_score,accuracy_score,recall_score



#二值化处理，传入np.array或list，输出二值化后的同类型数据
def binarize(result):
    for i in range(len(result)):
        if result[i]>=0.5:
            result[i]=1
        else:
            result[i]=0
    return result
#样例
#result = binarize(result)

#计算F1score的函数,传入预测结果和正确的结果，返回F1_score
def cal_f1(result_valid,lb_valid):
    TP,FP,FN = 0,0,0
    for i in range(len(lb_valid)):
        if result_valid[i] ==1 and lb_valid[i] == 1:
            TP +=1
        if result_valid[i]==1 and lb_valid[i]==0:
            FP +=1
        if result_valid[i] == 0 and lb_valid[i]==1:
            FN +=1
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    F1 = 2*precision*recall/(precision + recall)
    print('precision:',precision)
    print('recall:',recall)
    print('F1:',F1)
    return True
#样例
#cal_f1(result_valid,lb_valid)

#测试集预测,传入lgb_model、sid以及测试集特征，直接生成csv文件并返回True
#如果需要二值化，则多一条binarize = True
def predict(bst,sid,test,outputfile,binarize = False):
    result = bst.predict(test)
    if binarize==True:
        result = binarize(list(result))
    #进行拼接并生成最终文件
    result = pd.DataFrame(result)
    final = pd.concat([sid,result], axis=1, join='outer')
    final.columns = ['sid','label']
    final.to_csv(outputfile,index = None)
    return True
#使用样例
#predict(bst,sid,test,'lgb.csv',binarize = False)

#lgb训练log作图，传入metric_dict，自动生成图像，并在最后一个点处标记数据
def picture_lgb_loss(metric_dict):
    plt.figure(figsize=(8, 6))
    plt.plot(metric_dict['valid_0']['auc'])
    plt.plot(metric_dict['valid_0']['binary_logloss'])
    plt.title('lgb-naive log')
    plt.ylabel('Loss&AUC')
    plt.xlabel('Epoch')
    plt.legend(['val_auc','valid_loss'], loc='right')
    
    final = len(metric_dict['valid_0']['auc'])

    plt.text(final, metric_dict['valid_0']['auc'][final-1] ,"final:"+str(metric_dict['valid_0']['auc'][final-1])
             , ha='center', va='bottom', fontsize=10)
    plt.text(final, metric_dict['valid_0']['binary_logloss'][final-1] ,"final:"+str(metric_dict['valid_0']['binary_logloss'][final-1])
             , ha='center', va='bottom', fontsize=10)
    plt.show()
#样例
#picture_lgb_loss(metric_dict)    
    
# 创建训练集，验证集，本地训练集
def create_train_valid_test(train_x,train_y,untrain_size=0.1,random=0):
    #train_x，总训练特征，dataframe
    #train_y，总训练标签，Series
    #untrain_size，非训练集比例
    #random，随机划分的种子，种子相同即随机划分相同
    #划分训练集和非训练集
    train_X,valid_X, train_Y, valid_Y = train_test_split(train_x,
                                                   train_y,
                                                   test_size = untrain_size,
                                                   random_state = random)
    #将非训练集划分为验证集和本地测试集
    valid_X,test_X,valid_Y,test_Y = train_test_split(valid_X,
                                                   valid_Y,
                                                   test_size = 0.5,
                                                   random_state = random)
    return train_X,valid_X,test_X, train_Y, valid_Y,test_Y
#样例
#train_X,valid_X,local_test_X, train_Y, valid_Y,local_test_Y = create_train_valid_test(train_x,train_y)

# 创建可训练的lgb数据集
def create_lgb_dataset(data,categorical_feature=None,free_raw_data=True):
    # list of dataframe，格式：[train_X,valid_X,train_Y,valid_Y]
    # categorical_feature：哪些特征要被视为分类特征；注意使用，分类特征和数值特征的处理方法不同
    # free_raw_data：是否释放原始数据；当需要使用分类特征是，必须为False
    if categorical_feature != None:
        free_raw_data=False
    train_data = lgb.Dataset(data[0],data[2],categorical_feature=categorical_feature,free_raw_data=free_raw_data)
    valid_data = lgb.Dataset(data[1],data[3],categorical_feature=categorical_feature,free_raw_data=free_raw_data)
    #返回可训练的训练集与验证集
    return train_data,valid_data



#可用于early_stop,方式：bst = lgb.train 里面加一个 feval=lgb_f1
def lgb_f1_score(y_pred, data):
    y_true = data.get_label()  #lgb_data获取label
    y_pred = np.round(y_pred) # 默认精度为0，四舍五入
    return 'f1', f1_score(y_pred, y_true), True

In [20]:
#未加入初赛
%time data = pd.read_feather('feature/combine.bin', use_threads = 12)
data = data.drop(['sid'],axis=1)
##划分数据：
train=data[1000000:6000000]
label=train['label']

data = data.drop(['label'],axis = 1)
train=data[1000000:6000000]
test=data[6000000:].reset_index(drop=True)
del data

In [2]:
#加入初赛
%time data = pd.read_feather('feature/combine.bin', use_threads = 12)
data = data.drop(['sid'],axis=1)
##划分数据：
train=data[:6000000]
label=train['label']

data = data.drop(['label'],axis = 1)
train=data[:6000000]
test=data[6000000:].reset_index(drop=True)
del data

Wall time: 3.8 s


In [3]:
cols = train.columns

In [5]:
#lgb训练的模型,传入训练集、标签以及测试集,如果没有指定k折训练，则自动进行1折训练
#返回预测的结果
def lgb_model(train, label, test, params, k_split = 0, up_load = False):
    #如果不上交（本地lgb）
    if up_load == False:
        # 创建训练集，验证集，本地训练集
        train_X,valid_X,local_test_X, train_Y, valid_Y,local_test_Y = create_train_valid_test(train,label)
        # 创建可训练的lgb数据集
        train_data,valid_data = create_lgb_dataset([train_X,valid_X,train_Y,valid_Y])

        if k_split==0:
            #不分折训练
           
            #lgb训练， metric_dict保存训练产生指标
            metric_dict = {}
            bst = lgb.train(params, train_data,100000,valid_sets = valid_data,early_stopping_rounds=100,
                            verbose_eval = 200, callbacks = [lgb.record_evaluation(metric_dict)],feval=lgb_f1)

            #输出特征数
            print("特征数：",bst.num_feature())
            #保存lgb模型
            #bst.save_model('lgb.model')
            f_importance = bst.feature_importance(importance_type = 'gain')
            f_name = bst.feature_name()
            print("特征重要性(信息增益)：")
            for i in range(len(f_name)):
                print(f_name[i]+': '+str(f_importance[i]))
            #绘图——loss
            picture_lgb_loss(metric_dict)
            #评估
            cal_f1(binarize(bst.predict(local_test_X)),np.array(local_test_Y))
            result = bst.predict(test)
            return result

        #k折交叉训练
        if k_split!=0:

            #skf=StratifiedKFold(y,n_folds=5,shuffle=True,random_state=2018)
            #5折交叉训练分布  （skf确保训练集，测试集中各类别样本的比例与原始数据集中相同。即skf划分出来的训练集标签一定各个种类都有）
            skf = StratifiedKFold(n_splits=k_split,shuffle=True,random_state=98)

            ##用于存放测试集概率，k折最后要除以k取平均
            result = np.zeros(test.shape[0])  
            train_X = np.array(train_X)
            train_Y = np.array(train_Y)
            
            #对于每一折的训练，i计数，tr为训练集所代表的的行数列表，va为验证集所代表的的行数列表
            for i,(tr,va) in enumerate(skf.split(train_X,train_Y)):
                print(type(train))
                print(tr)
                print(train_X[tr])
                print('fold:',i+1,'training')
                dtrain = lgb.Dataset(train_X[tr],train_Y[tr])
                dvalid = lgb.Dataset(train_X[va],train_Y[va],reference=dtrain)
                ##训练：
                metric_dict = {}
                bst = lgb.train(params, dtrain, num_boost_round=30000, valid_sets=dvalid,
                                verbose_eval=400,early_stopping_rounds=200, callbacks = [lgb.record_evaluation(metric_dict)],
                               feval=lgb_f1)

                ##预测测试集：
                result += bst.predict(local_test_X, num_iteration=bst.best_iteration)
                
            result/=5
            cal_f1(binarize(rsult),np.array(local_test_Y))
            
            return result
    #如果上交，把全部数据丢入训练
    if up_load == True:
        if k_split == 0 :
            #上交的lgb（所有都拿来训练）
            train_data_full = lgb.Dataset(train,label)
            bst_full = lgb.train(params, train_data_full,3300)
            #输出特征数
            print("特征数：",bst_full.num_feature())
            #输出特征重要性
            print("特征重要性：",bst_full.feature_importance(importance_type = 'gain'))
            #保存lgb模型
            #bst_full.save_model('lgb_full.model')
            result = bst.predict(test)
            return result
        
        if k_split!=0:

            #skf=StratifiedKFold(y,n_folds=5,shuffle=True,random_state=2018)
            #5折交叉训练分布  （skf确保训练集，测试集中各类别样本的比例与原始数据集中相同。即skf划分出来的训练集标签一定各个种类都有）
            skf = StratifiedKFold(n_splits=k_split,shuffle=True,random_state=97)

            ##用于存放测试集概率，k折最后要除以k取平均
            result = np.zeros(test.shape[0])  
            train = np.array(train)
            label = np.array(label)
            #对于每一折的训练，i计数，tr为训练集所代表的的行数，va为验证集所代表的的行数
            for i,(tr,va) in enumerate(skf.split(train,label)):
                print('fold:',i+1,'training')
                dtrain = lgb.Dataset(train[tr],label[tr])
                dvalid = lgb.Dataset(train[va],label[va],reference=dtrain)
                ##训练：
                metric_dict = {}
                bst = lgb.train(params, dtrain, num_boost_round=5000, valid_sets=dvalid,
                                verbose_eval=100,early_stopping_rounds=200, callbacks = [lgb.record_evaluation(metric_dict)],
                                feval=lgb_f1_score)
                print("特征重要性：",list(zip(list(cols),list(bst.feature_importance(importance_type = 'gain')))))

                ##预测测试集：
                result += bst.predict(test, num_iteration=bst.best_iteration)
                
            result /= k_split
            return result
            
        
#本地运行不分折训练
#lgb_model(train,label,test, k_split = 0, up_load = False)

In [6]:

#默认参数
params1 = {'boosting_type': 'goss',
                          'objective': 'binary',
                          'metric': ['auc','binary_logloss'],

                          'num_leaves': 31,
                          #'max_depth' : 7,
                          'learning_rate': 0.05,
                          #'feature_fraction': 0.9,
                  }
#k折训练参数，有几个特征就写几个种子
params2 = {
            'learning_rate': 0.03,
            'boosting_type': 'goss',
            'objective': 'binary',
            'metric': 'auc',
            'feature_fraction': 0.8,
            #'bagging_fraction': 0.8,#使用goss，bagging-fraction需要为1
            'bagging_freq': 5,
            'num_leaves': 1000,
            'verbose': -1,
            'max_depth': -1,
          #  'reg_alpha':2.2,
          #  'reg_lambda':1.4,
            'seed':74,
            }

result = lgb_model(train,label,test,params2, k_split = 5, up_load = True)

fold: 1 training
Training until validation scores don't improve for 200 rounds.
[100]	valid_0's auc: 0.995049	valid_0's f1: 0.974375
[200]	valid_0's auc: 0.995857	valid_0's f1: 0.975689
[300]	valid_0's auc: 0.996134	valid_0's f1: 0.976311
[400]	valid_0's auc: 0.996225	valid_0's f1: 0.97654
[500]	valid_0's auc: 0.996266	valid_0's f1: 0.976673
[600]	valid_0's auc: 0.996301	valid_0's f1: 0.976775
[700]	valid_0's auc: 0.996323	valid_0's f1: 0.976831
[800]	valid_0's auc: 0.99634	valid_0's f1: 0.976872
[900]	valid_0's auc: 0.996357	valid_0's f1: 0.976908
[1000]	valid_0's auc: 0.996362	valid_0's f1: 0.976965
[1100]	valid_0's auc: 0.996373	valid_0's f1: 0.976995
[1200]	valid_0's auc: 0.996383	valid_0's f1: 0.977027
[1300]	valid_0's auc: 0.996399	valid_0's f1: 0.977074
[1400]	valid_0's auc: 0.996408	valid_0's f1: 0.977119
[1500]	valid_0's auc: 0.996419	valid_0's f1: 0.977159
[1600]	valid_0's auc: 0.996429	valid_0's f1: 0.977154
[1700]	valid_0's auc: 0.996431	valid_0's f1: 0.977214
[1800]	valid_

特征重要性： [('empty_count', 811760.4356360435), ('make', 1911286.143917203), ('model', 1420218.64465487), ('big_model', 470539.84404599667), ('model_equal_make', 308630.13609564304), ('ip', 371573.6116772294), ('ip1', 36984.154819488525), ('reqrealip', 496689.65310412645), ('ip2', 131877.17154628038), ('ip3', 108085.81123256683), ('reqrealip1', 66343.27381438017), ('reqrealip2', 85617.08725965023), ('reqrealip3', 60953.183020591736), ('ip_equal', 16727.220734119415), ('adidmd5', 476233.29062479734), ('imeimd5', 641477.0884170532), ('macmd5', 339068.1739528179), ('adunitshowid', 941824.4487285614), ('mediashowid', 15354019.33601731), ('city', 498878.82257300615), ('province', 19652.2474193573), ('h', 3598590.4671848416), ('w', 863821.065466702), ('ppi', 1511254.1555612087), ('orientation', 77227.3534039259), ('pkgname_count', 9516297.822720945), ('ver_count', 9242899.396773398), ('adunitshowid_count', 1642108.8206537962), ('mediashowid_count', 9559167.411546826), ('apptype_count', 4108792.9

[3390]	valid_0's auc: 0.99652	valid_0's f1: 0.977378
特征重要性： [('empty_count', 800306.4370956123), ('make', 1902827.65324077), ('model', 1366445.2759127617), ('big_model', 549443.186637789), ('model_equal_make', 333950.88115751743), ('ip', 422935.27622815967), ('ip1', 41161.237851053476), ('reqrealip', 518824.36276966333), ('ip2', 149405.96089339256), ('ip3', 120038.76184371114), ('reqrealip1', 64296.93212661147), ('reqrealip2', 82865.70783981681), ('reqrealip3', 57036.47600826621), ('ip_equal', 15165.188340723515), ('adidmd5', 549109.5793167353), ('imeimd5', 732123.7106039226), ('macmd5', 378275.3815316856), ('adunitshowid', 938994.0353797972), ('mediashowid', 15362618.73156631), ('city', 558606.6054040492), ('province', 21041.843602836132), ('h', 3609055.216380745), ('w', 835931.5747249424), ('ppi', 1501333.3434887528), ('orientation', 79815.78314214945), ('pkgname_count', 9450017.480844557), ('ver_count', 9315956.523910403), ('adunitshowid_count', 1655106.7658245564), ('mediashowid_co

In [7]:
result = binarize(np.array(result))

In [8]:
result = pd.DataFrame(result)
#生成答案
test_a = pd.read_csv('round2_test_a.txt','\t')
test_b = pd.read_csv('test_b.txt','\t')
test = pd.concat([test_a,test_b],axis=0,sort=False).reset_index(drop=True)

sid = pd.DataFrame(test['sid'])
final = pd.concat([sid,result], axis=1, join='outer')
final.columns = ['sid','label']
final.to_csv('lgb_5fold_try.csv',index = None)

FileNotFoundError: [Errno 2] File b'test_a.txt' does not exist: b'test_a.txt'